# Importing initial libraries

In [1]:

import datetime
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr

# Downloading and saving data

In [2]:
# Set up yfinance to use pandas data reader
yf.pdr_override()

# Define the list of tickers
tickers = ['COR', 'CE', 'HUM', 'CINF', 'KEYS', 'NXPI', 'BMY', 'EMR', 'CVS', 'RVTY', 'ES', 'DHI', 'ZBRA', 'KMI', 'INVH', 'GPC', 'SWKS', 'GIS']

# Define the date range
start_date = '2002-01-01'
end_date = '2022-12-31'
Output_file = 'stock_prices.xlsx'

# Create a pandas DataFrame to store the data
price_data = pdr.get_data_yahoo(tickers, start=start_date, end=end_date)['Close']
adj_price_data = pdr.get_data_yahoo(tickers, start=start_date, end=end_date)['Adj Close']
volume_data = pdr.get_data_yahoo(tickers, start=start_date, end=end_date)['Volume']

[*********************100%%**********************]  18 of 18 completed
[*********************100%%**********************]  18 of 18 completed
[*********************100%%**********************]  18 of 18 completed


In [3]:
# Convert the datetime index to string in the format "MM/DD/YYYY"
price_data.index = price_data.index.strftime('%m/%d/%Y')
adj_price_data.index = adj_price_data.index.strftime('%m/%d/%Y')
volume_data.index = volume_data.index.strftime('%m/%d/%Y')

In [4]:

# Create an Excel writer
with pd.ExcelWriter(Output_file, engine='openpyxl') as writer:
    # Save close prices to "Price_daily" sheet
    price_data.to_excel(writer, sheet_name='Price_daily')

    # Save adjusted close prices to "Adj_Price_daily" sheet
    adj_price_data.to_excel(writer, sheet_name='Adj_Price_daily')

    # Save volume data to "Volume_daily" sheet
    volume_data.to_excel(writer, sheet_name='Volume_daily')

print("Stock prices saved to stock_prices.xlsx")

Stock prices saved to stock_prices.xlsx


In [5]:
sp500_data = pdr.get_data_yahoo('^GSPC', start=start_date, end=end_date)[["Adj Close", "Close", "High", "Low", "Open", "Volume"]]
sp500_data.index = sp500_data.index.strftime('%m/%d/%Y')

with pd.ExcelWriter(Output_file, engine='openpyxl', mode= 'a', if_sheet_exists= "replace") as writer:
    sp500_data.to_excel(writer, sheet_name='S&P 500')
print("Prices for the S&P 500 index (^GSPC) saved to stock_prices.xlsx")

[*********************100%%**********************]  1 of 1 completed
Prices for the S&P 500 index (^GSPC) saved to stock_prices.xlsx


# Calculating Market Captilization

In [6]:
from Market_Cap import calculate_market_caps

# Load the S&P 500 constituents data from the Excel file
constituents_data = pd.read_excel("S&P 500 Constituent.xlsx", sheet_name="S&P 500 Constituent")

# Filter the relevant columns
constituents_data = constituents_data[["ticker", "Name", "Share_outstanding"]]

# Define other parameters
startsdate = datetime.datetime(2002, 1, 1)
endsdate = datetime.datetime(2022, 12, 31)

# Calculate market caps using the imported function
market_caps = calculate_market_caps(tickers, constituents_data, startsdate, endsdate)

# Create an Excel writer
with pd.ExcelWriter(Output_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    # Add a column for the date
    market_caps.insert(0, "Date", market_caps.index.strftime("%d-%m-%Y"))

    # Save the market caps to a new sheet
    market_caps.to_excel(writer, sheet_name=f"Market_Caps", index=False)

print("Market capitalization data added to the Excel file.")

Market capitalization data added to the Excel file.


# Making excel file more readalbe. 
###### (This is the last step)

#### Wraping Text

In [7]:
import openpyxl

# Load existing Excel file
file_path = 'stock_prices.xlsx'
book = openpyxl.load_workbook(file_path)

# Access all sheets in the workbook
for sheet_name in book.sheetnames:
    sheet = book[sheet_name]

    # Set text wrapping for all non-empty cells in the sheet
    for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
        for cell in row:
            if cell.value is not None:
                cell.alignment = openpyxl.styles.Alignment(wrap_text=True)

# Save the changes back to the Excel file
book.save(file_path)
book.close()

print("Text wrapping set for all non-empty cells in stock_prices.xlsx")


Text wrapping set for all non-empty cells in stock_prices.xlsx
